In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv('/kaggle/input/heart-attack-analysis-prediction-dataset/heart.csv')
features=data.columns[:-1]
data.head()

In [ ]:
data.shape

In [ ]:
data.info()
import seaborn as sns
sns.histplot(data,x="output")

In [ ]:
nulls = data.isna().sum()
nulls

# **Train XGBoost Model**

In [ ]:
# output variable dataframe
y = data.iloc[:,-1].values
y

In [ ]:

train = data.iloc[:,:-1].values
train

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.2, random_state=42)
print(X_test.shape)

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# random = RandomForestClassifier(n_estimators=100)
# random.fit(X_train,y_train)
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train, y_train)

# Predictions

In [ ]:
y_pred = xgb.predict(X_test)
y_pred

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

In [ ]:
# from sklearn.feature_extraction import DictVectorizer
# vec = DictVectorizer()
# booster = xgb.get_booster()
# # original_feature_names = booster.feature_names
# features = vec.get_feature_names()
# # print(booster.get_dump()[0])
# # recover original feature names
# # booster.feature_names = original_feature_names
# # original_feature_names
# features

# **Model Explainability/Interpretation with Eli5

In [ ]:
import eli5
from eli5 import formatters
from IPython.display import display
# eli5.show_weights(xgb)
display(eli5.explain_weights(xgb,feature_names = list(features),importance_type='gain')) #Once can replace gain with cover, weight; default is gain

In [ ]:
feature_importance = formatters.as_dataframe.explain_weights_df(xgb,feature_names = list(features),importance_type='gain')
feature_importance

In [ ]:
features = list(features)
features

In [ ]:
for i in range(len(X_test)):
    display(eli5.show_prediction(xgb,X_test[i],show_feature_values=True,feature_names=features))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
for i in range(len(X_test)):
    display(eli5.formatters.as_dataframe.explain_prediction_df(xgb,X_test[i],feature_names=features,target_names=['0','1']))

In [ ]:
# import lime
# from lime import lime_tabular
# exp = lime_tabular.LimeTabularExplainer(X_train,mode='classification',training_labels=features,)
# e = exp.explain_instance(X_test[60],xgb.predict_proba,num_features=13)
# e.as_list()